In [1]:
import os

exon_files_dir = "/s/project/first_last_exon/Data/output_data/HITindex/rna_seq/step_2/chr_files"
exon_files = [f for f in os.listdir(exon_files_dir) if f.endswith(".exon")]
print("We have {} different .exon files".format(len(exon_files)))

We have 100 different .exon files


In [2]:
# Read exon files into dataframes

import pandas as pd

dataframes_ls = []
for f in exon_files:
    df = pd.read_csv(os.path.join(exon_files_dir, f),  sep='\t')
    dataframes_ls.append(df)
    
merged_df = pd.concat(dataframes_ls, sort=False)
merged_df.head()

,exon,gene,strand,nTXPT,nFE,nINTERNAL,nLE,nSINGLE,nUP,nDOWN,...,PofIL,downstream_fraction,HIT_postmean,CI_75,CI_90,CI_95,pval_CI,pval_internal,ID,ID_position
0,chr1:15005-15038,ENSG00000227232.5,-,1,0,0,0,0,24,20,...,0.08036,0.49515,0.00969,"-0.091,0.273","-0.136,0.364","-0.182,0.364",0.892,0.321,internal,last
1,chr1:15796-15947,ENSG00000227232.5,-,1,0,0,0,0,4,57,...,0.00000,0.91973,-0.83947,"-0.934,-0.77","-0.934,-0.705","-0.967,-0.705",0.000,0.000,FirstInternal_high,FirstInternal_high
2,chr1:16607-16765,ENSG00000227232.5,-,1,0,0,0,0,1,4,...,0.03351,0.56797,-0.13594,"-1.0,0.2","-1.0,0.2","-1.0,0.6",0.412,0.216,internal,internal
3,chr1:16858-17055,ENSG00000227232.5,-,1,0,0,0,0,11,1,...,0.76961,0.18971,0.62059,"0.5,1.0","0.333,1.0","0.333,1.0",0.020,0.007,InternalLast_medium,InternalLast_medium
4,chr1:17233-17368,ENSG00000227232.5,-,1,0,0,0,0,91,9,...,0.99994,0.09892,0.80216,"0.74,0.88","0.7,0.9","0.68,0.92",0.000,0.000,InternalLast_high,InternalLast_high


In [3]:
genes_set = merged_df["gene"].unique()
classes_labels_set = ["first", "FirstInternal_medium", "FirstInternal_high", "last", "InternalLast_medium", "InternalLast_high", "internal"]

print("Total number of unique genes = {}".format(len(genes_set)))

Total number of unique genes = 1177


In [4]:
# seqname - name of the chromosome or scaffold; chromosome names can be given with or without the 'chr' prefix. Important note: the seqname must be one used within Ensembl, i.e. a standard chromosome name or an Ensembl identifier such as a scaffold ID, without any additional content such as species or assembly. See the example GFF output below.
# source - name of the program that generated this feature, or the data source (database or project name)
# feature - feature type name, e.g. Gene, Variation, Similarity
# start - Start position* of the feature, with sequence numbering starting at 1.
# end - End position* of the feature, with sequence numbering starting at 1.
# score - A floating point value.
# strand - defined as + (forward) or - (reverse).
# frame - One of '0', '1' or '2'. '0' indicates that the first base of the feature is the first base of a codon, '1' that the second base is the first base of a codon, and so on..
# attribute - A semicolon-separated list of tag-value pairs, providing additional information about each feature.

In [5]:
first_exons_merge_output = {"seqname": [],
               "source": [],
                "feature": [],
                "start": [],
                "end": [],
                "score": [],
                "strand": [],
                "frame": [],
                "attribute": []
                           }
last_exons_merge_output = {"seqname": [],
               "source": [],
                "feature": [],
                "start": [],
                "end": [],
                "score": [],
                "strand": [],
                "frame": [],
                "attribute": []
                          }
def add_entry(dic, gene, exon, strand):
#     chr1:s-e
    dic["seqname"].append("chr1")
    dic["source"].append("hitindex")
    dic["feature"].append("exon")
    start, end = exon.split(":")[1].split("-")
    dic["start"].append(start)
    dic["end"].append(end)
    dic["score"].append(".")
    dic["strand"].append(strand)
    dic["frame"].append(".")
    dic["attribute"].append("gene_id:"+ gene)

    
# Extract first and last exons records
for gene in genes_set:
    gene_rows = merged_df[merged_df["gene"] == gene]
    exons_set = gene_rows["exon"].unique()
    for exon in exons_set:
        labels_set = gene_rows[gene_rows["exon"] == exon]["ID_position"].unique()
        first = False
        last = False
        for l in labels_set:
            if "first" in l.lower():
                first = True
            elif "last" in l.lower():
                last = True
        if first:
            add_entry(first_exons_merge_output, gene, exon, gene_rows["strand"].unique()[0])
        if last:
            add_entry(last_exons_merge_output, gene, exon, gene_rows["strand"].unique()[0])
            
#         hybird = not (len(labels_set) == 1 and (("last" in labels_set or "first" in labels_set or "internal" in labels_set)))
#         exon_type = "hybird" if hybird else labels_set[0]
#         add_entry(merge_output, gene, exon, gene_rows["strand"].unique()[0], exon_type)
first_exons_merge_output_df = pd.DataFrame(first_exons_merge_output)
first_exons_merge_output_df.to_csv("first_exons_merge_output.gtf", index=False, sep='\t', header=False)

last_exons_merge_output_df = pd.DataFrame(last_exons_merge_output)
last_exons_merge_output_df.to_csv("last_exons_merge_output.gtf", index=False, sep='\t', header=False)